```mermaid
flowchart TD
    A(工艺及介质参数)-->D(凝固传热计算)
    B(钢种热物性参数及计算条件)-->D(凝固传热计算)
    C(设备参数)-->D(凝固传热计算)
    D-->E(连铸坯温度场)
```

title：连铸区温度场模拟计算
## 钢物性参数 tab
    col1 20%
        - 浇筑钢种 selectbox
        - 浇筑温度 number_input
        - 液相线温度 number_input
        - 固相线温度 number_input
        - 热导率 number_input
        - 比热容 number_input
        - 密度 number_input
    col2 80%
        - 各项参数的显示
## 工艺及设备参数 tab
    
    - col1 15% 结晶器
      - 拉坯速度 number_input
      - 热流密度修正系数 number_input
      - 断面宽度 number_input
      - 断面厚度 number_input
      - 钢液高度 number_input
    - col2 15% 二冷区
      - 分区数目 number_input
        - 每个分区的冷却水量
        - 每个分区的冷却水温度
    - col3 70% 相关参数显示
      - 二冷区结构图形
      - 各项冷却水的水量与冷却水温度
## 计算参数 tab
    - col1 15% 计算设置
      - 空间步长
      - 时间步长
      - 初始化
      - 计算
    - col2 85% 结果显示
      - 结晶器进出口热图
      - 各二冷区分区热图

# 控制方程的理解与推导
## 核心控制方程
$$\rho c_{p} \frac{\partial T}{\partial t}=\frac{\partial }{\partial x}(\lambda \frac{\partial T}{\partial x})+\frac{\partial }{\partial y}(\lambda \frac{\partial T}{\partial y})+\frac{\partial }{\partial z}(\lambda \frac{\partial T}{\partial z})+\dot{q}$$
## 假设
- 拉坯方向的传热可以忽略，$\frac{\partial T}{\partial z}=0$，只考虑二维情况.
- 内热源作等效比热容处理，$\dot{q}=0$.
- 冷却水温度与环境温度忽略不计.
- 模拟1/4边界
## 简化后的控制方程及离散化
$$\rho \frac{\partial H}{\partial t}=\frac{\partial }{\partial x}(\lambda \frac{\partial T}{\partial x})+\frac{\partial }{\partial y}(\lambda \frac{\partial T}{\partial y})$$
### 方程左侧离散化
$$(\rho \frac{\partial H}{\partial t})_{i}^{p}=\rho\frac{H_{i}^{p+1}-H_{i}^{p}}{\Delta t}$$
### 方程右侧离散化
$$(\frac{\partial }{\partial x}(\lambda \frac{\partial T}{\partial x}))_{i}^{p}=\frac{T_{i+1}^{p}-2T_{i}^{p}+T_{i-1}^{p}}{(\Delta x)^{2}}$$
$$(\frac{\partial }{\partial x}(\lambda \frac{\partial T}{\partial x}))_{j}^{p}=\frac{T_{j+1}^{p}-2T_{j}^{p}+T_{j-1}^{p}}{(\Delta y)^{2}}$$
最终简化后的离散方程可化为:
$$H_{i}^{p+1}=H_{i}^{p}+\frac{\lambda\Delta t}{\rho(\Delta x)^{2}}(T_{i+1}^{p}-2T_{i}^{p}+T_{i-1}^{p})+\frac{\lambda\Delta t}{\rho(\Delta y)^{2}}(T_{j+1}^{p}-2T_{j}^{p}+T_{j-1}^{p})$$
## 边界条件
1.当x=0或者y=0时,为绝热边界条件.即$T_{i-1}^{p}=T_{i}^{p}$或$T_{j-1}^{p}=T_{j}^{p}$,离散方程为:
### 左边界
$$H_{i}^{p+1}=H_{i}^{p}+\frac{\lambda\Delta t}{\rho(\Delta x)^{2}}(T_{i+1}^{p}-T_{i}^{p})+\frac{\lambda\Delta t}{\rho(\Delta y)^{2}}(T_{j+1}^{p}-2T_{j}^{p}+T_{j-1}^{p})$$
### 下边界
$$H_{i}^{p+1}=H_{i}^{p}+\frac{\lambda\Delta t}{\rho(\Delta x)^{2}}(T_{i+1}^{p}-2T_{i}^{p}+T_{i-1}^{p})+\frac{\lambda\Delta t}{\rho(\Delta y)^{2}}(T_{j+1}^{p}-T_{j}^{p})$$
### 右边界
x=width/2，y!=thickness/2，对流边界条件,需要在方程右边加上对流项$(-\frac{2h\Delta t(T_{i}^{p}-T_{f}^{p})}{\rho\Delta x})$,没有了i+1项
$$H_{i}^{p+1}=H_{i}^{p}-\frac{2h\Delta t}{\rho\Delta x}(T_{i}^{p}-T_{f}^{p})+\frac{\lambda\Delta t}{\rho(\Delta y)^{2}}(T_{j+1}^{p}-2T_{j}^{p}+T_{j-1}^{p})$$
### 上边界
x!=width/2,y=thickness/2，对流边界条件需要在方程右边加上对流项$(-\frac{2h\Delta t(T_{j}^{p}-T_{f}^{p})}{\rho\Delta y})$,没有了j+1项
$$H_{j}^{p+1}=H_{j}^{p}-\frac{2h\Delta t}{\rho\Delta y}(T_{j}^{p}-T_{f}^{p})+\frac{\lambda\Delta t}{\rho(\Delta x)^{2}}(T_{i+1}^{p}-2T_{i}^{p}+T_{i-1}^{p})$$
### 右上角角点
x=width/2，y=thickness/2，没有了j+1和i+1项，添加两个对流项
$$H_{i,j}^{p+1}=H_{i,j}^{p}-\frac{2\lambda\Delta t}{\rho\Delta y}(T_{i,j-1}^{p}-T_{i,j}^{p})-\frac{2\lambda\Delta t}{\rho\Delta x}(T_{i-1,j}^{p}-T_{i,j}^{p})+\frac{\lambda\Delta t}{\rho(\Delta x)^{2}}(T_{i+1,j}^{p}-2T_{i,j}^{p}+T_{i-1,j}^{p})+\frac{\lambda\Delta t}{\rho(\Delta y)^{2}}(T_{i,j+1}^{p}-2T_{i,j}^{p}+T_{i,j-1}^{p})$$
### 说明
- $h(T_{i,j}-T_{f})可简化为热流密度\phi $